# Getting the data

## Step 1: Import Libraries
We'll use the following libraries in this notebook:

In [1]:
import pandas as pd
from datetime import datetime, timedelta, time
import numpy as np
from scipy.ndimage.interpolation import shift
from collections import defaultdict
import seaborn as sns
import sys

%matplotlib inline

In [2]:
print("Python Version:", sys.version)
print("Pandas Version:", pd.__version__)
print("Numpy Version:", np.__version__)
print("Seaborn Version:", sns.__version__)

Python Version: 3.6.3 |Anaconda custom (64-bit)| (default, Oct 13 2017, 12:02:49) 
[GCC 7.2.0]
Pandas Version: 0.22.0
Numpy Version: 1.13.3
Seaborn Version: 0.8.0


## Step 2: Getting the right data 
We pulled in weekly turnstile data from the MTA portal: http://web.mta.info/developers/turnstile.html

First, we create a list of the weeks we're interested in fetching data for

In [3]:
# Define list of weeks we want to pull from the MTA portal

def datelist(startdate):
    """
    For a given Saturday, make a list of dates for the 14 previous Saturdays
    """
    week_list = [startdate + ((timedelta(days=-7))*i) for i in range(14)]
    clean_weeks = [i.strftime('%y%m%d') for i in week_list]
    return clean_weeks


# Define the last Saturday we're interested in for 2016 and 2017
start17 = datetime(2017, 7, 1)
start16 = datetime(2016, 7, 2)

# We'll import data for the 14 weeks preceeding July 1st for both 2016 and 2017
weeks_to_import = datelist(start17) + datelist(start16)
weeks_to_import

['170701',
 '170624',
 '170617',
 '170610',
 '170603',
 '170527',
 '170520',
 '170513',
 '170506',
 '170429',
 '170422',
 '170415',
 '170408',
 '170401',
 '160702',
 '160625',
 '160618',
 '160611',
 '160604',
 '160528',
 '160521',
 '160514',
 '160507',
 '160430',
 '160423',
 '160416',
 '160409',
 '160402']

We then iterate through our list of dates to pull weekly files from the MTA portal

In [4]:
def loadturndata(date):
    # Build the filename
    strdate = str(date)
    filename = 'http://web.mta.info/developers/data/nyct/turnstile/turnstile_'+strdate+'.txt'

    # Read in the csv
    df = pd.read_csv(filename)
    return df


def loadturnlist(dates):
    """
    We'll iterarte through the list of weeks to create dataframes using loadturndata and then concat together into one dataframe 
    """
    data = pd.DataFrame()
    x = []
    for i in dates:
        df = (loadturndata(i))
        x.append(df)
    data = pd.concat(x)
    return data

In [5]:
# Note: This takes a few minutes to run - go treat yourself to a cup of tea!
raw = loadturnlist(weeks_to_import)

KeyboardInterrupt: 

In [ ]:
# Pickle the raw data in case things go south in the cleaning process and you need to start over from here
raw.to_pickle('data/raw_turnstile_data.pkl')

# Cleaning the data
All the fun stuff (jk!)

## Rename columns and add datetime columns

In [6]:
# Uncomment below to read in the pickled raw data if you are starting here
raw = pd.read_pickle('data/raw_turnstile_data.pkl')

In [7]:
# Rename columns
df = raw.rename(columns=lambda x: x.strip().lower())

# Concat date and time and convert to datetime object
df['datetime'] = df['date'] + ' ' + df['time']
df['datetime_clean'] = [datetime.strptime(x, '%m/%d/%Y %H:%M:%S') for x in df['datetime']]

In [8]:
# Add some helpful date-part columns
df['year'] = [x.year for x in df['datetime_clean']]
df['weekday'] = df[['datetime_clean']].apply(lambda x: datetime.strftime(x['datetime_clean'], '%A'), axis=1)

In [9]:
df.head()

,c/a,unit,scp,station,linename,division,date,time,desc,entries,exits,datetime,datetime_clean,year,weekday
0,A002,R051,02-00-00,59 ST,NQR456W,BMT,06/24/2017,00:00:00,REGULAR,6233682,2110437,06/24/2017 00:00:00,2017-06-24 00:00:00,2017,Saturday
1,A002,R051,02-00-00,59 ST,NQR456W,BMT,06/24/2017,04:00:00,REGULAR,6233696,2110445,06/24/2017 04:00:00,2017-06-24 04:00:00,2017,Saturday
2,A002,R051,02-00-00,59 ST,NQR456W,BMT,06/24/2017,08:00:00,REGULAR,6233712,2110473,06/24/2017 08:00:00,2017-06-24 08:00:00,2017,Saturday
3,A002,R051,02-00-00,59 ST,NQR456W,BMT,06/24/2017,12:00:00,REGULAR,6233790,2110560,06/24/2017 12:00:00,2017-06-24 12:00:00,2017,Saturday
4,A002,R051,02-00-00,59 ST,NQR456W,BMT,06/24/2017,16:00:00,REGULAR,6233942,2110622,06/24/2017 16:00:00,2017-06-24 16:00:00,2017,Saturday


## Find delta counts for distinct turnstiles at given time intervals

In [10]:
# Create group ID for distinct turnstiles
df['group'] = df['c/a'].astype(str) + \
                df['unit'].astype(str) + \
                df['scp'].astype(str) + \
                df['station'].astype(str)  + \
                df['linename'].astype(str) + \
                df['division'].astype(str) + \
                df['year'].astype(str)
                
# Map 'group' string to integer id     
groups = set(df['group'])


def groups_dict(groups):
    group_dict = defaultdict(int)
    for i in enumerate(list(groups)):
        group_dict[i[1]]= i[0]

    return group_dict

group_id_dict = groups_dict(groups)

df['group_id'] = [group_id_dict[x] for x in df['group']]

Create station ID for later grouping on distinct stations

In [11]:
# Create station ID for distinct stations
df['station_line'] = df['station'].astype(str) + \
                df['linename'].astype(str)

Sort values in dataframe by group id and datatime to find diff in counts from prev row

In [12]:
# Sort values by group id and date to find diff in turnstile counts from prev row
df.sort_values(['group_id','datetime_clean'], inplace=True)
df.reset_index(drop=True)

def find_diff_prev_row(df_series_col):
    col_array = np.array(df_series_col)
    col_array_shifted = shift(col_array, 1, cval=np.NaN)
    col_diff = abs(col_array - col_array_shifted)

    return col_diff


df['entries_diff'] = find_diff_prev_row(df['entries'])
df['exit_diff'] = find_diff_prev_row(df['exits'])

Set invalid diff values to nan (first row of turnstile partitions and negative values from reboots)

In [13]:
# Identify first rows for each group partition to use as mask when setting invalid values to nan
def find_first_rows_groups(df_series_col):
    col_array = np.array(df_series_col)
    col_array_shifted = shift(col_array, 1, cval=np.NaN)
    first_row_mask = col_array != col_array_shifted

    return first_row_mask


df['first_row_group'] = find_first_rows_groups(df['group_id'])

# Make entries_diff and exit_diff nan when first row in group or negative value
df.loc[df['first_row_group'], 'entries_diff'] = None
df.loc[df['entries_diff'] < 0, 'entries_diff'] = None

df.loc[df['first_row_group'], 'exit_diff'] = None
df.loc[df['exit_diff'] < 0, 'exit_diff'] = None

## Deal with outliers and missing values

We'll call the describe method to check out the distribution of data for the entry and exit diffs calculated above

In [14]:
df.describe()

,entries,exits,year,group_id,entries_diff,exit_diff
count,5.481151e+06,5.481151e+06,5.481151e+06,5.481151e+06,5.471787e+06,5.471787e+06
mean,3.668874e+07,2.942573e+07,2.016504e+03,4.681123e+03,1.098745e+04,1.020665e+04
std,1.995975e+08,1.789708e+08,4.999825e-01,2.701979e+03,3.712465e+06,3.654583e+06
min,0.000000e+00,0.000000e+00,2.016000e+03,0.000000e+00,0.000000e+00,0.000000e+00
25%,5.403665e+05,2.665350e+05,2.016000e+03,2.344000e+03,1.100000e+01,9.000000e+00
50%,2.597810e+06,1.490638e+06,2.017000e+03,4.680000e+03,8.200000e+01,5.500000e+01
75%,6.628317e+06,4.688856e+06,2.017000e+03,7.019000e+03,2.580000e+02,1.730000e+02
max,2.147483e+09,2.097170e+09,2.017000e+03,9.363000e+03,2.130766e+09,2.097170e+09


### Outliers
Outliers are commonly defined as values that fall above 1.5 IQR from the 75th Q. If we use this definition, ~13% of our values would qualify as outliers

In [15]:
def find_outliers(df_series, multiple_IQR):
    """
    For a series of numerical values, remove the zeros and identify the upper outliers 
    to return a mask for all outliers in series
    """
    non_zeros = df_series.replace(0, None)
    
    adjusted_IQR = (non_zeros.quantile(.75) - non_zeros.quantile(.25)) * multiple_IQR
    outlier_lim = non_zeros.quantile(.75) + adjusted_IQR
    print(outlier_lim)
    
    outliers = [True if x > outlier_lim else False for x in df_series]
    
    outlier_count = sum(outliers)
    all_data_count = len(df_series)
    print('{} outliers identified: {} of all data'.format(outlier_count, round(outlier_count/all_data_count,6)))
    
    return outliers

In [16]:
print('Entries Outliers')
df['entries_outlier'] = find_outliers(df['entries_diff'], 5)

print('\n Exit Outliers')
df['exit_outlier'] = find_outliers(df['exit_diff'], 5)

Entries Outliers
1564.0
21642 outliers identified: 0.003948 of all data

 Exit Outliers
1041.0
69320 outliers identified: 0.012647 of all data


In [17]:
print('All Data Len:', len(df))

clean_df = df.loc[(~df['entries_outlier'])].copy()
print('Excluding Outliers Len:', len(clean_df))

print('Keeping', round(len(clean_df)/len(df), 6))

All Data Len: 5481151
Excluding Outliers Len: 5459509
Keeping 0.996052


### Missing Values
We've decided to remove null values from our dataset

In [18]:
print('Null entry diffs', clean_df.entries_diff.isnull().sum())
print('Null exit diffs', clean_df.exit_diff.isnull().sum())
print('Clean Data len:', len(clean_df))

Null entry diffs 9364
Null exit diffs 9364
Clean Data len: 5459509


In [19]:
clean_df.dropna(subset = ['entries_diff', 'exit_diff'], how='any', inplace=True)

print('Null entry diffs', clean_df.entries_diff.isnull().sum())
print('Null exit diffs', clean_df.exit_diff.isnull().sum())
print('Clean Data len:', len(clean_df))

Null entry diffs 0
Null exit diffs 0
Clean Data len: 5450145


In [20]:
thrown_away = len(df) - len(clean_df)
print("We're throwing away {} data points - about {} of the total".format(thrown_away, round(thrown_away/len(df), 4)))

We're throwing away 31006 data points - about 0.0057 of the total


# Explore the distributions

## Add columns for aggregating & exploring distributions


In [21]:
clean_df['week'] = [x.isocalendar()[1] for x in clean_df['datetime_clean']]## Find average daily entry volume by station
clean_df['hour'] = [x.hour for x in clean_df['datetime_clean']]

We'll want to bin the times in 4 hour increments and rename the hour groups and weekday 

In [22]:
def timebin(hour):
    if hour ==0:
        return 6
    if hour <= 4:
        return 1
    if hour <=8:
        return 2
    if hour <=12:
        return 3
    if hour <= 16:
        return 4
    if hour <= 20:
        return 5
    if hour <= 24:
        return 6
    
hourgroups = {6:'8pm - 12am', 
              1: '12am - 4am', 
              2:'4am - 8am', 
              3:'8am - 12pm', 
              4:'12pm - 4pm', 
              5:'4pm - 8pm'}

wkdaynbr = {'Friday': 5,
 'Monday': 1,
 'Saturday': 6,
 'Sunday': 0,
 'Thursday': 4,
 'Tuesday': 2,
 'Wednesday': 3}

In [23]:
clean_df['timegroup'] = clean_df['hour'].apply(timebin)
clean_df['timegroupstr'] = clean_df['timegroup'].map(hourgroups)
clean_df['wkdaynbr'] = clean_df['weekday'].map(wkdaynbr)

In [24]:
clean_df.head()

,c/a,unit,scp,station,linename,division,date,time,desc,entries,...,entries_diff,exit_diff,first_row_group,entries_outlier,exit_outlier,week,hour,timegroup,timegroupstr,wkdaynbr
97614,N523,R300,00-00-00,2 AV,F,IND,03/25/2017,04:00:00,REGULAR,16253671,...,251.0,151.0,False,False,False,12,4,1,12am - 4am,6
97615,N523,R300,00-00-00,2 AV,F,IND,03/25/2017,08:00:00,REGULAR,16253760,...,89.0,134.0,False,False,False,12,8,2,4am - 8am,6
97616,N523,R300,00-00-00,2 AV,F,IND,03/25/2017,12:00:00,REGULAR,16254010,...,250.0,492.0,False,False,False,12,12,3,8am - 12pm,6
97617,N523,R300,00-00-00,2 AV,F,IND,03/25/2017,16:00:00,REGULAR,16254463,...,453.0,787.0,False,False,False,12,16,4,12pm - 4pm,6
97618,N523,R300,00-00-00,2 AV,F,IND,03/25/2017,20:00:00,REGULAR,16254958,...,495.0,1192.0,False,False,True,12,20,5,4pm - 8pm,6


Now we can find daily average entry traffic for each station

Picked the cleaned turnstile dataset for joining with station data! 

In [25]:
clean_df.to_pickle('data/cleaned_turnstile_data.pkl')

## Find average daily entry volume by station

In [ ]:
# Find daily average entries per station
stations_day = clean_df.groupby(['station_line', 'date']).sum()
stations_day.reset_index(inplace=True)

daily_avg = stations_day.groupby('station_line')['entries_diff'].mean()
daily_avg.sort_values(ascending=False, inplace=True)

In [ ]:
sns.distplot(daily_avg, hist=True, kde=True);

In [ ]:
daily_avg.head(10)

In [ ]:
clean_df.head()